In [ ]:
import pyodbc
import xml.etree.ElementTree as ET

In [1]:
pip install pyodbc

In [ ]:
def connect_to_db():
    connection_string = """
    Driver={ODBC Driver 17 for SQL Server};
    Server=YOUR_SERVER_NAME;
    Database=aboriDatabase;
    User Id=YOUR_USER_ID;
    Password=YOUR_PASSWORD;
    """
    return pyodbc.connect(connection_string)

def insert_data_to_db(data, connection):
    cursor = connection.cursor()

    # Inserting Journal data
    cursor.execute("""
        INSERT INTO dbo.journal (title, abbrev_title, issn, publisher_name)
        VALUES (?, ?, ?, ?)
    """, data["journal_title"], data["abbrev_journal_title"], data["issn"], data["publisher_name"])
    journal_id = cursor.execute("SELECT SCOPE_IDENTITY()").fetchone()[0]

    # Inserting Article data
    cursor.execute("""
        INSERT INTO dbo.article (journal_id, title, keywords)
        VALUES (?, ?, ?)
    """, journal_id, data["article_title"], ', '.join(data["keywords"]))
    article_id = cursor.execute("SELECT SCOPE_IDENTITY()").fetchone()[0]

    # Inserting Authors and linking them to Articles
    for author in data["authors"]:
        # Here I'm assuming you'd have a function to fetch researcher_id by the author name
        # If not, you'd have to insert the researcher into the researcher table first
        researcher_id = get_or_create_researcher_id(author, cursor)
        cursor.execute("""
            INSERT INTO dbo.article_authors (article_id, researcher_id)
            VALUES (?, ?)
        """, article_id, researcher_id)

    # Inserting Full Text data
    cursor.execute("""
        INSERT INTO dbo.article_fulltext (article_id, content)
        VALUES (?, ?)
    """, article_id, data["full_text"])

    # Committing changes to the database
    connection.commit()

def get_or_create_researcher_id(name, cursor):
    # Split name into first and last
    first_name, last_name = name.split(' ', 1)
    cursor.execute("""
        SELECT researcher_id FROM dbo.researcher WHERE name = ?
    """, name)
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        cursor.execute("""
            INSERT INTO dbo.researcher (name)
            VALUES (?)
        """, name)
        return cursor.execute("SELECT SCOPE_IDENTITY()").fetchone()[0]

if __name__ == "__main__":
    xml_path = 'path_to_your_xml.xml'
    data = extract_data_from_xml(xml_path)
    connection = connect_to_db()
    insert_data_to_db(data, connection)
    connection.close()
